## Train, save and infer CNN classifier

## import some important libraries

In [22]:
import torch
import torchvision
import torchvision.transforms as transforms

## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Making train and test Transforms

In [74]:
train_transforms = transforms.Compose([
    transforms.Resize((32,32)),
    #transforms.RandomHorizontalFlip(p = 0.25),
    #transforms.RandomRotation(10), #10 degree rotation
    transforms.ToTensor(),# necessary for pytorch
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    #transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))


In [75]:
test_transforms = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()
])

In [76]:
batch_size = 4

## Creat train and test data Loader

In [77]:
training_dataset_path = "./drive/MyDrive/training/training"

In [78]:
testing_dataset_path = './drive/MyDrive/validation/validation'

In [79]:
Train_Ds =torchvision.datasets.ImageFolder(root = training_dataset_path, transform = train_transforms)
Test_Ds =torchvision.datasets.ImageFolder(root = testing_dataset_path, transform = test_transforms)

In [80]:
train_loader = torch.utils.data.DataLoader(dataset = Train_Ds, batch_size = 4, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = Test_Ds, batch_size = 4, shuffle = False)

## Classes

In [81]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Build a Neural Network

In [82]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [83]:
net = Net()

## Define optimizer and Loss function

In [84]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Start Training of CNN

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')


## Save Model information

In [86]:
PATH = './drive/MyDrive/bestModel_final2.pth'
torch.save(net.state_dict(), PATH)

## Load saved model

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH))

In [88]:
dataiter = iter(test_loader)
images, labels = next(dataiter)

In [89]:
outputs = net(images)


Check model accuracy on Validation dataset

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the  test images: {100 * correct // total} %')

## Inference of learned model on single image

In [91]:
classes = [
    "a","b","c","d","e","f","g","h","i","k"
]

In [92]:
images_transforms = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),# necessary for pytorch
])

In [95]:
from PIL import Image

In [96]:
def classifier(model, images_transforms, image_path, classes):
  model = model.eval()
  image = Image.open(image_path)
  image = images_transforms(image).float()
  image = image.unsqueeze(0)

  output = model(image)
  _, predicted = torch.max(output.data, 1)
  print(classes[predicted.item()])

In [ ]:
classifier(net, images_transforms, "./drive/MyDrive/validation/validation/n1/n100.jpg", classes)
